In [ ]:
# call data.fif
import mne
import numpy as np

raw = mne.io.read_raw_fif('data.fif', preload=True)


In [ ]:
print(raw.info)

In [ ]:
print(raw.ch_names)

In [ ]:

raw.annotations

In [ ]:
events, events_id = mne.events_from_annotations(raw)
print(events_id)

In [ ]:
print(events)

In [ ]:
raw.load_data()

In [ ]:
raw.filter(.5, 30., fir_design='firwin')

picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False,
                       exclude='bads')


tmin, tmax = 0, 2
event_id = dict({'fist': 1, 'hook': 2, 'index': 3, 'open': 4, 'thumb': 5})

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)

In [ ]:
evoked = epochs['fist'].average()
print(evoked)
evoked.plot(time_unit='s')

evoked = epochs['hook'].average()
print(evoked)
evoked.plot(time_unit='s')

evoked = epochs['index'].average()
print(evoked)
evoked.plot(time_unit='s')

evoked = epochs['open'].average()
print(evoked)
evoked.plot(time_unit='s')

evoked = epochs['thumb'].average()
print(evoked)
evoked.plot(time_unit='s')

In [ ]:
# Getting labels and changing labels from 7,8,9,10 -> 1,2,3,4
labels = epochs.events[:,-1]
data = epochs.get_data()
print(data.shape)

In [ ]:
import pywt

# signal is decomposed to level 5 with 'db4' wavelet

def wpd(X): 
    coeffs = pywt.WaveletPacket(X,'db4',mode='symmetric',maxlevel=5)
    return coeffs
             
def feature_bands(x):
    
    Bands = np.empty((8,x.shape[0],x.shape[1],38)) # 8 freq band coefficients are chosen from the range 4-32Hz
    
    for i in range(x.shape[0]):
        for ii in range(x.shape[1]):
             pos = []
             C = wpd(x[i,ii,:]) 
             pos = np.append(pos,[node.path for node in C.get_level(5, 'natural')])
             for b in range(1,9):
                 Bands[b-1,i,ii,:] = C[pos[b]].data
        
    return Bands

wpd_data = feature_bands(data)
print(wpd_data.shape)

In [ ]:
from mne.decoding import CSP 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


# OneHotEncoding Labels
enc = OneHotEncoder()
X_out = enc.fit_transform(labels.reshape(-1,1)).toarray()
print(X_out)
# Cross Validation Split
cv = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 0)
print(cv)

In [ ]:
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from mne.decoding import CSP

svms = []
Csp = [CSP(n_components=4, reg=None, log=True, norm_trace=False) for _ in range(8)]
Csp = [Csp[x].fit(wpd_data[x,...],labels) for x  in range(8)]
ss = preprocessing.StandardScaler()
ss = ss.fit(np.concatenate(tuple(Csp[x].transform(wpd_data[x, ...]) for x  in range(8)),axis=-1))

acc = []
ka = []
prec = []
recall = []

for train_idx, test_idx in cv.split(labels):
    
    label_train, label_test = labels[train_idx], labels[test_idx]
    y_train, y_test = X_out[train_idx], X_out[test_idx]
    
    # Extraction features 
    X_train = ss.transform(np.concatenate(tuple(Csp[x].transform(wpd_data[x,train_idx,:,:]) for x  in range(8)),axis=-1))

    X_test = ss.transform(np.concatenate(tuple(Csp[x].transform(wpd_data[x,test_idx,:,:]) for x  in range(8)),axis=-1))
    
    # Build and train SVM classifier
    svm = SVC(kernel='linear', C=1, probability=True)
    svm.fit(X_train, label_train)

    svms.append(svm)
    
    # Predict on test data
    y_pred = svm.predict(X_test)
    y_prob = svm.predict_proba(X_test)
    
    pred = (y_prob == y_prob.max(axis=1)[:, None]).astype(int)
    
    # Calculate and store metrics
    acc.append(accuracy_score(label_test, y_pred))
    ka.append(cohen_kappa_score(label_test, y_pred))
    prec.append(precision_score(label_test, y_pred, average='weighted'))
    recall.append(recall_score(label_test, y_pred, average='weighted'))


In [ ]:
import pandas as pd

scores = {'Accuracy':acc,'Kappa':ka,'Precision':prec,'Recall':recall}

Es = pd.DataFrame(scores)

avg = {'Accuracy':[np.mean(acc)],'Kappa':[np.mean(ka)],'Precision':[np.mean(prec)],'Recall':[np.mean(recall)]}

Avg = pd.DataFrame(avg)


T = pd.concat([Es,Avg])

T.index = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','Avg']
T.index.rename('Fold',inplace=True)

print(T)